# Feature Selection techniques discrimination

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tqdm import tqdm
from sklearn.metrics import classification_report
from statistics import mean
import warnings
warnings.filterwarnings('ignore')

hp = pd.read_csv("hp_waitTime_6vars.csv")

In [2]:
#Fucntion to split into X and Y
def getXandY(df):
    df.drop(df.tail(20).index,inplace=True) 
    x = df.drop(['Harry_Potter_and_the_Forbidden'],axis=1)
    y = df.Harry_Potter_and_the_Forbidden
    return(x,y)

#Function to split in test and train
def trainTest(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)
    return(X_train, X_test, y_train, y_test)

#Function to perform oversampling
def overSampling(X_train, y_train, y_test, method):
    X_train_os, y_train_os= method.fit_resample(X_train, y_train)
    # Check the number of records after over sampling
    #print(sorted(Counter(y_train_os).items())) 
    return(X_train_os, y_train_os)

In [4]:
#Variable order in manual correlation:
pear_corr = ['temperature','holiday','day','month','pressure','day.1',
            'report','minute','year','humidity','hour',	'Pandemic']
kend_corr = ['temperature','holiday','month','day','pressure','day.1','report','year',
             'minute','humidity','hour','Pandemic']
mutInf_class = ['month','day','year','hour','minute','holiday','day.1','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_reg = ['month','day','year','hour','minute','holiday','day.1','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_class2 = ['day','temperature','month','humidity','hour','day.1','pressure',
                 'holiday','year','report','minute','Pandemic']
varThre = ['month','day','year','hour','minute','holiday','day.1',
           'temperature','humidity','pressure','report']
mrmr = ['temperature','day.1','hour','holiday','humidity','day',
        'year','month','pressure','minute','report']
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']

In [5]:
order=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

In [5]:
#Split the compact dataframe into X and Y and train and test
x,y=getXandY(hp)
X_train, X_test, y_train, y_test = trainTest(x,y)
smote = SMOTE(random_state=42)
X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)

In [6]:
# Order of variables according to method:
order=[]

In [7]:
X_train_os_r = X_train_os[['temperature',	'holiday',	'day',	'month',	'pressure',	'day.1',
                           'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']]
X_train_os_r.head()

,temperature,holiday,day,month,pressure,day.1,report,minute,year,humidity,hour,Pandemic
0,0.828358,0.0,0.733333,0.363636,0.48,0.000000,0.0625,0.2,1.0,0.197531,0.608696,0.0
1,0.269851,0.0,0.633333,0.090909,0.72,1.000000,0.0625,0.6,1.0,0.703704,0.347826,0.0
2,0.700896,0.0,0.466667,0.454545,0.32,0.333333,0.3750,0.8,1.0,0.753086,0.391304,0.0
3,0.742388,0.0,0.800000,0.000000,0.32,0.166667,0.1250,0.8,1.0,0.432099,0.565217,0.0
4,0.626269,0.0,0.400000,0.909091,0.36,0.833333,0.3750,1.0,0.0,0.728395,0.739130,0.0


In [8]:
X_test_r = X_test[['temperature',	'holiday',	'day',	'month',	'pressure',	'day.1',
                           'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']]

In [9]:
df1= X_train_os_r.iloc[:, 0:2]
print(df1.columns)
lg = LogisticRegression(max_iter=20000)
lg.fit(df1, y_train_os).decision_function(X_test_r.iloc[:, 0:2])
y_pred=lg.predict(X_test.iloc[:, 0:2])

Index(['temperature', 'holiday'], dtype='object')


In [10]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.precision_score(y_test, y_pred,average='macro'))
print(metrics.recall_score(y_test, y_pred,average='macro'))
print(metrics.f1_score(y_test, y_pred, average='macro'))

0.018208765615075163
0.0026012522307250234
0.14285714285714285
0.0051094673677331195


In [6]:
from statistics import mean

In [7]:
smote = SMOTE(random_state=42)

def testModel(df,var_order,n_vars):
    for j in tqdm(range(1,n_vars)):
        #split our dataframe into X and Y
        x,y=getXandY(df)
        #create the lists to store metrics
        acc = []
        rec = []
        preci = []
        f1 = []
        for i in range(100):
            #split the dataFrame into test and train
            X_train, X_test, y_train, y_test = trainTest(x,y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[var_order]
            X_test_r = X_test[var_order]
            df1= X_train_os_r.iloc[:, 0:j] #use only part of the variables
            lg = LogisticRegression(max_iter=20000)
            lg.fit(df1, y_train_os).decision_function(X_test_r.iloc[:, 0:j])
            y_pred=lg.predict(X_test_r.iloc[:, 0:j])
            ac=metrics.accuracy_score(y_test, y_pred)
            acc.append(ac)
            p=metrics.precision_score(y_test, y_pred,average='macro')
            preci.append(p)
            r=metrics.recall_score(y_test, y_pred,average='macro')
            rec.append(r)
            f=metrics.f1_score(y_test, y_pred, average='macro')
            f1.append(f)
        print(df1.columns)
        print("For {} features: \n Accuracy: {} \n Precision: {} \n Recall: {} \n F1 score: {}".format(
        j,mean(acc),mean(preci),mean(rec),mean(f1)))
        print(classification_report(y_test, y_pred))

In [23]:
order=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

for i in range(len(order)):
    print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
    print('The variable order is: \n {}'.format(order[i]))
    testModel(hp,order[i],12)
    print('\n \n')

  0%|          | 0/11 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'day.1', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  9%|▉         | 1/11 [00:15<02:32, 15.21s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.2525682411583195 
 Precision: 0.17324067578623503 
 Recall: 0.17416228627318744 
 F1 score: 0.1250907062255531
              precision    recall  f1-score   support

        10.0       0.46      0.51      0.48      1517
        20.0       0.19      0.08      0.11       671
        30.0       0.16      0.02      0.04       408
        40.0       0.13      0.61      0.21       428
        50.0       0.00      0.00      0.00        61
        60.0       0.17      0.01      0.02       674
       100.0       0.11      0.01      0.02       454

    accuracy                           0.26      4213
   macro avg       0.17      0.18      0.13      4213
weighted avg       0.26      0.26      0.22      4213



 18%|█▊        | 2/11 [00:39<03:04, 20.51s/it]

...

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure'],
      dtype='object')
For 9 features: 
 Accuracy: 0.30374900504112495 
 Precision: 0.24059498299639906 
 Recall: 0.24884690390932207 
 F1 score: 0.22333237059651007
              precision    recall  f1-score   support

        10.0       0.60      0.45      0.52      1388
        20.0       0.26      0.13      0.18       626
        30.0       0.12      0.20      0.15       319
        40.0       0.15      0.16      0.15       370
        50.0       0.04      0.23      0.07        71
        60.0       0.25      0.17      0.20       559
       100.0       0.25      0.40      0.31       436

    accuracy                           0.30      3769
   macro avg       0.24      0.25      0.23      3769
weighted avg       0.36      0.30      0.31      3769



 91%|█████████ | 10/11 [09:53<01:28, 88.90s/it]

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.3044220037204358 
 Precision: 0.23957055817587833 
 Recall: 0.24460763207861885 
 F1 score: 0.22200151078717195
              precision    recall  f1-score   support

        10.0       0.62      0.48      0.54      1450
        20.0       0.23      0.10      0.13       584
        30.0       0.11      0.19      0.14       336
        40.0       0.16      0.13      0.15       321
        50.0       0.02      0.19      0.04        47
        60.0       0.30      0.17      0.21       595
       100.0       0.26      0.50      0.35       430

    accuracy                           0.31      3763
   macro avg       0.24      0.25      0.22      3763
weighted avg       0.37      0.31      0.33      3763



100%|██████████| 11/11 [11:52<00:00, 64.76s/it]

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.3101623635879691 
 Precision: 0.25005684223780034 
 Recall: 0.25547522195786837 
 F1 score: 0.2299079031260152
              precision    recall  f1-score   support

        10.0       0.63      0.46      0.53      1378
        20.0       0.26      0.15      0.19       571
        30.0       0.13      0.19      0.16       331
        40.0       0.11      0.05      0.07       369
        50.0       0.03      0.25      0.05        61
        60.0       0.26      0.18      0.22       628
       100.0       0.24      0.45      0.31       419

    accuracy                           0.30      3757
   macro avg       0.24      0.25      0.22      3757
weighted avg       0.36      0.30      0.32      3757


 



### One Hot Encoding

In [4]:
hp_oHe = pd.read_csv("harryPotterCleanOHE.csv")

In [5]:
#Delete all rows which contain outliers
hp_clean = hp_oHe[hp_oHe.Harry_Potter_and_the_Forbidden != 0] #delete rows with 0 min
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 180]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 150]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 145]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 135]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 130]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 11]
hp_fin_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 125]

#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_fin_clean.Harry_Potter_and_the_Forbidden.replace([5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 120, 110, 150,80, 90, 50,70], 
                                                       [10,10, 20, 30, 40, 50, 60, 60, 100, 100, 100, 100, 100, 100, 100,100, 100, 60,100])
#Create the final dataframe
df=pd.DataFrame(a)
hp2=hp_fin_clean.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3=pd.concat([hp2, df], axis=1)
hp4=hp3.drop('Unnamed: 0',axis=1)
hp4.drop(hp4.tail(20).index,inplace=True) 
hp4.Harry_Potter_and_the_Forbidden.unique()

array([ 40.,  20.,  10.,  30.,  60., 100.,  50.])

In [6]:
#Variable order in one hot encoding:
pear_corrO = ['temperature',	'holiday',	'day',	'month',	'pressure',	'day.1',	'shower rain',	'broken clouds',	'fog',	'overcast clouds',	'heavy intensity rain',	'minute',	'haze',	'thunderstorm with light rain',	'scattered clouds',	'clear sky',	'mist',	'light intensity drizzle',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'year',	'light rain',	'humidity',	'hour',	'Pandemic']
kend_corrO = ['temperature',	'holiday',	'month',	'day',	'pressure',	'day.1',	'shower rain',	'broken clouds',	'heavy intensity rain',	'fog',	'year',	'overcast clouds',	'minute',	'haze',	'thunderstorm with light rain',	'mist',	'light intensity drizzle',	'scattered clouds',	'clear sky',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'light rain',	'humidity',	'hour',	'Pandemic']
mutInf_classO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_regO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_class2O = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','day.1',	'year',	'holiday',	'shower rain',	'light rain',	'thunderstorm',	'fog',	'broken clouds',	'Pandemic',	'thunderstorm with rain',	'light intensity drizzle',	'thunderstorm with light rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds',	'clear sky']
varThreO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',]
mrmrO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds']

orderOHE=[pear_corrO,kend_corrO,mutInf_classO,mutInf_regO,mutInf_class2O,varThreO,mrmrO]

method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'varianceThreshold','MRMR']

In [16]:
def analizeDF(df,order,n_vars):
    for i in range(len(order)):
        print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
        print('The variable order is: \n {}'.format(order[i]))
        testModel(df,order[i],n_vars)
        print('\n \n')

In [ ]:
analizeDF(hp4,orderOHE)

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'day.1', 'shower rain', 'broken clouds', 'fog', 'overcast clouds', 'heavy intensity rain', 'minute', 'haze', 'thunderstorm with light rain', 'scattered clouds', 'clear sky', 'mist', 'light intensity drizzle', 'few clouds', 'thunderstorm', 'very heavy rain', 'moderate rain', 'thunderstorm with rain', 'year', 'light rain', 'humidity', 'hour', 'Pandemic']


  4%|▎         | 1/27 [00:36<15:54, 36.70s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.17324953468659895 
 Precision: 0.17324953468659895 
 Recall: 0.17324953468659895 
 F1 score: 0.17324953468659895
              precision    recall  f1-score   support

        10.0       0.44      0.55      0.49      1452
        20.0       0.19      0.08      0.12       813
        30.0       0.11      0.03      0.04       427
        40.0       0.12      0.03      0.05       525
        50.0       0.02      0.55      0.04        83
        60.0       0.14      0.01      0.03       777
       100.0       0.16      0.02      0.03       466

    accuracy                           0.21      4543
   macro avg       0.17      0.18      0.11      4543
weighted avg       0.24      0.21      0.19      4543



  7%|▋         | 2/27 [01:18<16:29, 39.58s/it]

Index(['temperature', 'holiday'], dtype='object')
For 2 features: 
 Accuracy: 0.18821769592790738 
 Precision: 0.18821769592790738 
 Recall: 0.18821769592790738 
 F1 score: 0.18821769592790738
              precision    recall  f1-score   support

        10.0       0.45      0.49      0.47      1515
        20.0       0.22      0.14      0.17       807
        30.0       0.12      0.01      0.02       434
        40.0       0.14      0.20      0.16       518
        50.0       0.03      0.36      0.05        88
        60.0       0.00      0.00      0.00       730
       100.0       0.24      0.18      0.21       445

    accuracy                           0.24      4537
   macro avg       0.17      0.20      0.15      4537
weighted avg       0.24      0.24      0.23      4537



...

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm'],
      dtype='object')
For 17 features: 
 Accuracy: 0.25605117923044296 
 Precision: 0.25605117923044296 
 Recall: 0.25605117923044296 
 F1 score: 0.25605117923044296
              precision    recall  f1-score   support

        10.0       0.62      0.48      0.54      1411
        20.0       0.23      0.12      0.16       548
        30.0       0.13      0.21      0.16       301
        40.0       0.18      0.14      0.16       319
        50.0       0.02      0.20      0.03        49
        60.0       0.30      0.17      0.22       581
       100.0       0.29      0.43      0.35       428

    accuracy                           0.31      3637
   macro avg       0.25      0.25      0.23      3637
weighted avg       0.38      0.

 67%|██████▋   | 18/27 [22:45<14:32, 96.98s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain'],
      dtype='object')
For 18 features: 
 Accuracy: 0.25545272935229185 
 Precision: 0.25545272935229185 
 Recall: 0.25545272935229185 
 F1 score: 0.25545272935229185
              precision    recall  f1-score   support

        10.0       0.60      0.46      0.52      1374
        20.0       0.27      0.16      0.20       581
        30.0       0.11      0.17      0.13       316
        40.0       0.11      0.08      0.09       306
        50.0       0.02      0.18      0.04        51
        60.0       0.28      0.19      0.23       549
       100.0       0.27      0.42      0.33       454

    accuracy                           0.30      3631
   macro avg       0.24      0.24      0.22      3631
weighted avg    

 70%|███████   | 19/27 [24:37<13:30, 101.33s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain'],
      dtype='object')
For 19 features: 
 Accuracy: 0.25625414423739423 
 Precision: 0.25625414423739423 
 Recall: 0.25625414423739423 
 F1 score: 0.25625414423739423
              precision    recall  f1-score   support

        10.0       0.64      0.52      0.58      1388
        20.0       0.29      0.13      0.18       576
        30.0       0.11      0.18      0.14       281
        40.0       0.18      0.16      0.17       342
        50.0       0.03      0.18      0.04        62
        60.0       0.28      0.19      0.23       565
       100.0       0.26      0.41      0.32       411

    accuracy                           0.33      3625
   macro avg       0.25      0.25      0.24      3625
weighted avg    

In [11]:
orderOHE=[mrmrO]
method = ['MRMR']
analizeDF(hp4,orderOHE)

------------------------- Analyzing method MRMR -------------------------
The variable order is: 
 ['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1', 'temperature', 'humidity', 'pressure', 'heavy intensity rain', 'light rain', 'broken clouds', 'scattered clouds', 'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain', 'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain', 'overcast clouds', 'moderate rain', 'minute', 'haze', 'few clouds']


  4%|▎         | 1/27 [01:10<30:22, 70.11s/it]

Index(['month'], dtype='object')
For 1 features: 
 Accuracy: 0.14419970062139495 
 Precision: 0.14419970062139495 
 Recall: 0.14419970062139495 
 F1 score: 0.14419970062139495
              precision    recall  f1-score   support

        10.0       0.34      0.52      0.41      1497
        20.0       0.18      0.07      0.10       825
        30.0       0.00      0.00      0.00       468
        40.0       0.00      0.00      0.00       568
        50.0       0.00      0.00      0.00        76
        60.0       0.00      0.00      0.00       777
       100.0       0.14      0.59      0.23       500

    accuracy                           0.24      4711
   macro avg       0.09      0.17      0.11      4711
weighted avg       0.15      0.24      0.17      4711



  7%|▋         | 2/27 [02:21<29:30, 70.83s/it]

Index(['month', 'day'], dtype='object')
For 2 features: 
 Accuracy: 0.16147886760258875 
 Precision: 0.16147886760258875 
 Recall: 0.16147886760258875 
 F1 score: 0.16147886760258875
              precision    recall  f1-score   support

        10.0       0.34      0.37      0.36      1468
        20.0       0.15      0.05      0.07       796
        30.0       0.09      0.09      0.09       459
        40.0       0.00      0.00      0.00       541
        50.0       0.00      0.00      0.00        89
        60.0       0.18      0.21      0.20       838
       100.0       0.15      0.41      0.22       514

    accuracy                           0.21      4705
   macro avg       0.13      0.16      0.13      4705
weighted avg       0.19      0.21      0.19      4705



...

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute'],
      dtype='object')
For 25 features: 
 Accuracy: 0.2533507947974572 
 Precision: 0.2533507947974572 
 Recall: 0.2533507947974572 
 F1 score: 0.2533507947974572
              precision    recall  f1-score   support

        10.0       0.56      0.51      0.53      1483
        20.0       0.17      0.11      0.13       735
        30.0       0.12      0.14      0.12       465
        40.0       0.21      0.14      0.17       523
        50.0       0.03      0.24      0.06        86
        60.0       0.28      0.07      0.11       761
       100.0       0.24      0.48      0.32       51

 96%|█████████▋| 26/27 [34:15<01:49, 109.76s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze'],
      dtype='object')
For 26 features: 
 Accuracy: 0.25262392386944676 
 Precision: 0.25262392386944676 
 Recall: 0.25262392386944676 
 F1 score: 0.25262392386944676
              precision    recall  f1-score   support

        10.0       0.59      0.55      0.57      1519
        20.0       0.22      0.15      0.18       743
        30.0       0.12      0.11      0.11       444
        40.0       0.25      0.15      0.18       521
        50.0       0.04      0.33      0.08        89
        60.0       0.26      0.08      0.12       762
       100.0       0.23      0.48      0

100%|██████████| 27/27 [36:12<00:00, 80.45s/it] 

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'day.1',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze', 'few clouds'],
      dtype='object')
For 27 features: 
 Accuracy: 0.2538590321504077 
 Precision: 0.2538590321504077 
 Recall: 0.2538590321504077 
 F1 score: 0.2538590321504077
              precision    recall  f1-score   support

        10.0       0.58      0.51      0.54      1518
        20.0       0.22      0.15      0.18       749
        30.0       0.13      0.10      0.11       453
        40.0       0.26      0.20      0.23       532
        50.0       0.04      0.34      0.07        73
        60.0       0.25      0.09      0.13       758
       100.0       0.24      0

### Ordinal Encoding

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tqdm import tqdm
from sklearn.metrics import classification_report
import warnings
from statistics import mean
warnings.filterwarnings('ignore')

In [9]:
hp_OE = pd.read_csv("harryPotterCleanOE.csv")

In [10]:
#Variable order in ordinal encoding:
pear_corrOE = ['temperature',	'holiday',	'day',	'month',	'pressure',	'day.1',	'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']
kend_corrOE = ['temperature',	'holiday',	'month',	'day',	'pressure',	'day.1',	'report',	'year',	'minute',	'humidity',	'hour',	'Pandemic',]
mutInf_classOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_regOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_class2OE = ['day',	'temperature',	'month',	'humidity',	'hour',	'day.1',	'pressure',	'holiday',	'year',	'report',	'minute',	'Pandemic',]
varThreOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'day.1',	'temperature',	'humidity',	'pressure',	'report']
mrmrOE = ['temperature',	'day.1',	'hour',	'holiday',	'humidity',	'day',	'year',	'month',	'pressure',	'minute',	'report']

orderOE=[pear_corrOE,kend_corrOE,mutInf_classOE,mutInf_regOE,mutInf_class2OE,varThreOE,mrmrOE]

In [11]:
#Delete all rows which contain outliers
hp_clean = hp_OE[hp_OE.Harry_Potter_and_the_Forbidden != 0] #delete rows with 0 min
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 180]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 150]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 145]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 135]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 130]
hp_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 11]
hp_fin_clean = hp_clean[hp_clean.Harry_Potter_and_the_Forbidden != 125]

#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_fin_clean.Harry_Potter_and_the_Forbidden.replace([5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 120, 110, 150,80, 90, 50,70], 
                                                       [10,10, 20, 30, 40, 50, 60, 60, 100, 100, 100, 100, 100, 100, 100,100, 100, 60,100])
#Create the final dataframe
df=pd.DataFrame(a)
hp2=hp_fin_clean.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3=pd.concat([hp2, df], axis=1)
hp4=hp3.drop('Unnamed: 0',axis=1)
hp4.drop(hp4.tail(20).index,inplace=True) 
hp4.Harry_Potter_and_the_Forbidden.unique()

array([ 40.,  20.,  10.,  30.,  60., 100.,  50.])

In [17]:
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']
smote = SMOTE(random_state=42)
analizeDF(hp4,orderOE,12)

  0%|          | 0/11 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'day.1', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  9%|▉         | 1/11 [00:24<04:06, 24.61s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.22256518973924105 
 Precision: 0.16630941915159828 
 Recall: 0.185141269137034 
 F1 score: 0.11901379425631604
              precision    recall  f1-score   support

        10.0       0.43      0.53      0.47      1513
        20.0       0.19      0.10      0.13       830
        30.0       0.09      0.03      0.04       449
        40.0       0.15      0.04      0.07       548
        50.0       0.02      0.55      0.04        83
        60.0       0.11      0.01      0.02       795
       100.0       0.14      0.01      0.01       499

    accuracy                           0.21      4717
   macro avg       0.16      0.18      0.11      4717
weighted avg       0.23      0.21      0.19      4717



 18%|█▊        | 2/11 [00:56<04:22, 29.15s/it]

...

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure'],
      dtype='object')
For 9 features: 
 Accuracy: 0.2955464544816288 
 Precision: 0.23165552775785936 
 Recall: 0.24899882142840324 
 F1 score: 0.21212428730980465
              precision    recall  f1-score   support

        10.0       0.57      0.51      0.54      1481
        20.0       0.19      0.09      0.13       696
        30.0       0.10      0.11      0.11       387
        40.0       0.18      0.16      0.17       467
        50.0       0.05      0.31      0.09        83
        60.0       0.24      0.11      0.15       713
       100.0       0.20      0.42      0.27       446

    accuracy                           0.29      4273
   macro avg       0.22      0.24      0.21      4273
weighted avg       0.32      0.29      0.29      4273



 91%|█████████ | 10/11 [11:03<01:34, 94.38s/it]

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.2941481134286384 
 Precision: 0.23137749321081674 
 Recall: 0.24968256058223556 
 F1 score: 0.2122600018625916
              precision    recall  f1-score   support

        10.0       0.57      0.50      0.53      1419
        20.0       0.20      0.08      0.12       718
        30.0       0.12      0.16      0.14       439
        40.0       0.17      0.11      0.13       456
        50.0       0.04      0.26      0.07        78
        60.0       0.28      0.13      0.18       711
       100.0       0.21      0.46      0.29       446

    accuracy                           0.29      4267
   macro avg       0.23      0.24      0.21      4267
weighted avg       0.32      0.29      0.29      4267



100%|██████████| 11/11 [13:02<00:00, 71.16s/it] 

Index(['temperature', 'day.1', 'hour', 'holiday', 'humidity', 'day', 'year',
       'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.2976531330673551 
 Precision: 0.23246335112411404 
 Recall: 0.25072712792756774 
 F1 score: 0.21360757293433927
              precision    recall  f1-score   support

        10.0       0.57      0.53      0.55      1462
        20.0       0.20      0.11      0.14       725
        30.0       0.14      0.19      0.16       402
        40.0       0.19      0.11      0.14       443
        50.0       0.03      0.24      0.05        68
        60.0       0.30      0.11      0.16       716
       100.0       0.24      0.48      0.32       445

    accuracy                           0.30      4261
   macro avg       0.24      0.25      0.22      4261
weighted avg       0.34      0.30      0.30      4261


 

